<a href="https://colab.research.google.com/github/TresDtres/Remove_Wildcards_modifier/blob/main/Spanish_Lora_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⭐ Entrenador de Lora de Hollowstrawberry

Basado en el trabajo de [Kohya_ss](https://github.com/kohya-ss/sd-scripts) y [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb#scrollTo=-Z4w3lfFKLjr). ¡Gracias!

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning inference.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Origen** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |

In [1]:
import os
import re
import toml
import shutil
import zipfile
from time import time
from IPython.display import Markdown, display

if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "override_config_file" not in globals():
  override_config_file = None
if "continue_from_lora" not in globals():
  continue_from_lora = ""
if "weighted_captions" not in globals():
  weighted_captions = False
if "adjust_tags" not in globals():
  adjust_tags = False
if "keep_tokens_weight" not in globals():
  keep_tokens_weight = 1.0

COLAB = True # low ram
COMMIT = "5050971ac687dca70ba0486a583d283e8ae324e2"
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True

#@title ## 🚩 Empieza Aquí

#@markdown ### ▶️ Base
#@markdown El nombre de tu proyecto también es el nombre de la carpeta donde irán tus imágenes. No se permiten espacios.
nombre_proyecto = "jeunji" #@param {type:"string"}
project_name = nombre_proyecto
#@markdown La estructura de carpetas no importa y es por comodidad. Asegúrate de siempre elegir la misma. Me gusta organizar por proyecto.
estructura_de_carpetas = "Organizar por proyecto (MyDrive/Loras/nombre_proyecto/dataset)" #@param ["Organizar por categoría (MyDrive/lora_training/datasets/nombre_proyecto)", "Organizar por proyecto (MyDrive/Loras/nombre_proyecto/dataset)"]
folder_structure = estructura_de_carpetas
#@markdown Decidir el modelo base de entrenamiento. Los modelos por defecto producen los resultados más limpios y consistentes. Puedes cambiarlo por un modelo propio si lo deseas.
modelo_de_entrenamiento = "Anime (animefull-final-pruned-fp16.safetensors)" #@param ["Anime (animefull-final-pruned-fp16.safetensors)", "AnyLora (AnyLoRA_noVae_fp16-pruned.ckpt)"]
opcional_enlace_a_modelo_propio = "https://civitai.com/api/download/models/11732" #@param {type:"string"}
modelo_propio_basado_en_sd2 = False #@param {type:"boolean"}
custom_model_is_based_on_sd2 = modelo_propio_basado_en_sd2

if opcional_enlace_a_modelo_propio:
  model_url = opcional_enlace_a_modelo_propio
elif "AnyLora" in modelo_de_entrenamiento:
  model_url = "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AnyLoRA_noVae_fp16-pruned.ckpt"
else:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/animefull-final-pruned-fp16.safetensors"

#@markdown ### ▶️ Procesamiento <p>
#@markdown La resolución de 512 es estándar en Stable Diffusion 1.5. No es necesario recortar o achicar, el proceso es automático.
resolucion = 512 #@param {type:"slider", min:512, max:1024, step:128}
resolution = resolucion
#@markdown Esta opción va a voltear tus imágenes para así tener el doble y aprender mejor. <p>
#@markdown **Desactiva esto si te importan los elementos asimétricos en tu Lora.**
flip_aug = True #@param {type:"boolean"}
#markdown Leave empty for no captions.
caption_extension = ".txt" #param {type:"string"}
#@markdown Mezclar las tags de anime ayuda al aprendizaje. Una tag de activación va al inicio de cada archivo de texto y no se mezclará.
mezclar_tags = True #@param {type:"boolean"}
shuffle_caption = mezclar_tags
tags_de_activacion = "1" #@param [0,1,2,3]
keep_tokens = int(tags_de_activacion)

#@markdown ### ▶️ Pasos <p>
#@markdown Tus imágenes se repetirán este número de veces durante el entrenamiento. Recomiendo que el valor total sea entre 200 y 400.
num_repeats = 20 #@param {type:"number"}
#@markdown Cuánto tiempo deseas entrenar. Un buen punto de partida puede ser alrededor de 10 epochs o alrededor de 2000 pasos. <p>
#@markdown Un epoch es una cantidad de pasos igual a: tu cantidad de imágenes multipliccada por sus repeticiones, y dividido en el batch size.
unidad_preferida = "Epochs" #@param ["Epochs", "Pasos"]
cuantos = 10 #@param {type:"number"}
max_train_epochs = cuantos if unidad_preferida == "Epochs" else None
max_train_steps = cuantos if unidad_preferida == "Pasos" else None
#@markdown Guardar más epochs te permitirá comparar mejor el progreso de tu Lora.
guardar_cada_cuantos_epochs = 1 #@param {type:"number"}
save_every_n_epochs = guardar_cada_cuantos_epochs
guardar_solo_ultimos_epochs = 10 #@param {type:"number"}
keep_only_last_n_epochs = guardar_solo_ultimos_epochs
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs
#@markdown Un batch size mayor hace el entrenamiento más rápido, pero puede empeorar el aprendizaje. Se recomienda 2 o 3.
batch_size = 2 #@param {type:"slider", min:1, max:8, step:1}
train_batch_size = batch_size

#@markdown ### ▶️ Aprendizaje
#@markdown La tasa de aprendizaje es lo más importante. Si deseas entrenar más lento con muchas imágenes, o si tienes un dim y alpha altos, usa un unet de 2e-4 o menor. <p>
#@markdown El text encoder ayuda a tu Lora a aprender conceptos un poco mejor. Se recomienda la mitad o un quinto del unet. Puedes dejarlo en 0 para algunos estilos.
aprendizaje_unet = 5e-4 #@param {type:"number"}
unet_lr = aprendizaje_unet
aprendizaje_text_encoder = 1e-4 #@param {type:"number"}
text_encoder_lr = aprendizaje_text_encoder
#@markdown El scheduler es el algoritmo matemático que guiará el entrenamiento. Para personajes recomiendo `cosine_with_restarts` con un valor de 3. Si no estás seguro ponlo en `constant` e ignora el valor.
scheduler = "cosine_with_restarts" #@param ["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"]
lr_scheduler = scheduler
valor_de_scheduler = 3 #@param {type:"number"}
lr_scheduler_number = valor_de_scheduler
lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0
#@markdown Pasos de calentamiento durante el entrenamiento para un inicio eficiente. Recomiendo dejarlo en 5%.
calentamiento = 0.05 #@param {type:"slider", min:0.0, max:0.5, step:0.01}
lr_warmup_ratio = calentamiento
lr_warmup_steps = 0
#@markdown Nueva función que hace el aprendizaje mucho más eficiente. Puede que tus Loras estén listos en la mitad de epochs. Se usará un valor de 5.0 como en la [investigación](https://arxiv.org/abs/2303.09556).
min_snr_gamma = True #@param {type:"boolean"}
min_snr_gamma_value = 5.0 if min_snr_gamma else None

#@markdown ### ▶️ Estructura
#@markdown LoRA es el tipo clásico, mientras que LoCon es bueno con estilos. Lycoris requiere [esta extensión](https://github.com/KohakuBlueleaf/a1111-sd-webui-lycoris) para funcionar como un lora normal. Más información [aquí](https://github.com/KohakuBlueleaf/Lycoris).
lora_type = "LoRA" #@param ["LoRA", "LoCon Lycoris", "LoHa Lycoris"]

#@markdown Aquí hay algunos valores recomendados para las configuraciones de abajo:

#@markdown | type | network_dim | network_alpha | conv_dim | conv_alpha |
#@markdown | :---: | :---: | :---: | :---: | :---: |
#@markdown | LoRA | 32 | 16 |   |   |
#@markdown | LoCon | 16 | 8 | 8 | 1 |
#@markdown | LoHa | 8 | 4 | 4 | 1 |

#@markdown Un dim mayor aumenta el tamaño del Lora, puede aprender más pero no siempre es mejor. Se recomienda un dim entre 8 y 32, y un alpha igual a la mitad del dim.
network_dim = 16 #@param {type:"slider", min:1, max:128, step:1}
network_alpha = 8 #@param {type:"slider", min:1, max:128, step:1}
#@markdown Los siguientes valores no afectan a LoRA. Funcionan como el dim/alpha pero para las capas adicionales de los Lycoris.
conv_dim = 8 #@param {type:"slider", min:1, max:64, step:1}
conv_alpha = 1 #@param {type:"slider", min:1, max:64, step:1}
conv_compression = False #@param {type:"boolean"}

network_module = "lycoris.kohya" if "Lycoris" in lora_type else "networks.lora"
network_args = None if lora_type == "LoRA" else [
  f"conv_dim={conv_dim}",
  f"conv_alpha={conv_alpha}",
]
if "Lycoris" in lora_type:
  network_args.append(f"algo={'loha' if 'LoHa' in network_args else 'lora'}")
  network_args.append(f"disable_conv_cp={str(not conv_compression)}")

#markdown ### ▶️ Experimental
#markdown Save additional data equaling ~1 GB allowing you to resume training later.
save_state = False #param {type:"boolean"}
#markdown Resume training if a save state is found.
resume = False #param {type:"boolean"}

#@markdown ### ▶️ Listo
#@markdown Ahora puedes correr esta celda apretando el botón circular a la izquierda. ¡Buena suerte!


# 👩‍💻 Cool code goes here

root_dir = "/content" if COLAB else "~/Loras"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")

if "/Loras" in folder_structure:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
  log_folder    = os.path.join(main_dir, "_logs")
  config_folder = os.path.join(main_dir, project_name)
  images_folder = os.path.join(main_dir, project_name, "dataset")
  output_folder = os.path.join(main_dir, project_name, "output")
else:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
  images_folder = os.path.join(main_dir, "datasets", project_name)
  output_folder = os.path.join(main_dir, "output", project_name)
  config_folder = os.path.join(main_dir, "config", project_name)
  log_folder    = os.path.join(main_dir, "log")

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

def clone_repo():
  os.chdir(root_dir)
  !git clone https://github.com/kohya-ss/sd-scripts {repo_dir}
  os.chdir(repo_dir)
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/requirements.txt -q -O requirements.txt

def install_dependencies():
  clone_repo()
  !apt -y update -qq
  !apt -y install aria2
  !pip -q install --upgrade -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118

  # patch kohya for minor stuff
  if COLAB:
    !sed -i "s@cpu@cuda@" library/model_util.py # low ram
  if LOAD_TRUNCATED_IMAGES:
    !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  if BETTER_EPOCH_NAMES:
    !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
    !sed -i 's/model_name + "."/model_name + "-{:02d}.".format(num_train_epochs)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"

def validate_dataset():
  global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens, keep_tokens_weight, weighted_captions, adjust_tags
  supported_types = (".png", ".jpg", ".jpeg")

  print("\n💿 Revisando archivos...")
  if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
    print("💥 Error: Por favor elije un nombre de proyecto válido.")
    return

  if custom_dataset:
    try:
      datconf = toml.loads(custom_dataset)
      datasets = [d for d in datconf["datasets"][0]["subsets"]]
    except:
      print(f"💥 Error: ¡Tu configuración de datos propia es inválida o tiene errores! Por favor revisa el ejemplo original.")
      return
    reg = [d for d in datasets if d.get("is_reg", False)]
    for r in reg:
      print("📁"+r["image_dir"].replace("/content/drive/", "") + " (Regularización)")
    datasets = [d for d in datasets if d not in reg]
    datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
    folders = datasets_dict.keys()
    files = [f for folder in folders for f in os.listdir(folder)]
    images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
  else:
    folders = [images_folder]
    files = os.listdir(images_folder)
    images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

  for folder in folders:
    if not os.path.exists(folder):
      print(f"💥 Error: La carpeta {folder.replace('/content/drive/', '')} no existe.")
      return
  for folder, (img, rep) in images_repeats.items():
    if not img:
      print(f"💥 Error: La carpeta {folder.replace('/content/drive/', '')} está vacía.")
      return
  for f in files:
    if not f.lower().endswith(".txt") and not f.lower().endswith(supported_types):
      print(f"💥 Error: Archivo inválido encontrado: \"{f}\". Abortando.")
      return

  if not [txt for txt in files if txt.lower().endswith(".txt")]:
    caption_extension = ""
  if continue_from_lora and not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
    print(f"💥 Error: Archivo de continuar_lora inválido. Ejemplo: /content/drive/MyDrive/Loras/ejemplo.safetensors")
    return

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  total_steps = int(max_train_epochs*steps_per_epoch)
  lr_warmup_steps = int(total_steps * lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", ""))
    print(f"📈 Se encontraron {img} imágenes con {rep} repeticiones, equivalente a {img*rep} pasos.")
  print(f"📉 Divide {pre_steps_per_epoch} pasos en {train_batch_size} batch size para obtener {steps_per_epoch} pasos por epoch.")
  print(f"🔮 Habrá {max_train_epochs} epochs, para un total de alrededor de {total_steps} pasos totales.")

  if total_steps > 10000:
    print("💥 Error: Tus pasos totales on muy altos. Probablemente cometiste un error. Abortando...")
    return

  if adjust_tags:
    print(f"\n📎 Weighted tags: {'ON' if weighted_captions else 'OFF'}")
    if weighted_captions:
      print(f"📎 Will use {keep_tokens_weight} weight on {keep_tokens} activation tag(s)")
    print("📎 Adjusting tags...")
    adjust_weighted_tags(folders, keep_tokens, keep_tokens_weight, weighted_captions)

  return True

def adjust_weighted_tags(folders, keep_tokens: int, keep_tokens_weight: float, weighted_captions: bool):
  weighted_tag = re.compile(r"\((.+?):[.\d]+\)(,|$)")
  for folder in folders:
    for txt in [f for f in os.listdir(folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(folder, txt), 'r') as f:
        content = f.read()
      # reset previous changes
      content = content.replace('\\', '')
      content = weighted_tag.sub(r'\1\2', content)
      if weighted_captions:
        # re-apply changes
        content = content.replace(r'(', r'\(').replace(r')', r'\)').replace(r':', r'\:')
        if keep_tokens_weight > 1:
          tags = [s.strip() for s in content.split(",")]
          for i in range(min(keep_tokens, len(tags))):
            tags[i] = f'({tags[i]}:{keep_tokens_weight})'
          content = ", ".join(tags)
      with open(os.path.join(folder, txt), 'w') as f:
        f.write(content)

def create_config():
  global dataset_config_file, config_file, model_file

  if resume:
    resume_points = [f.path for f in os.scandir(output_folder) if f.is_dir()]
    resume_points.sort()
    last_resume_point = resume_points[-1] if resume_points else None
  else:
    last_resume_point = None

  if override_config_file:
    config_file = override_config_file
    print(f"\n⭕ Usando configuración propia {config_file}")
  else:
    config_dict = {
      "additional_network_arguments": {
        "unet_lr": unet_lr,
        "text_encoder_lr": text_encoder_lr,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": True if text_encoder_lr == 0 else None,
        "network_weights": continue_from_lora if continue_from_lora else None
      },
      "optimizer_arguments": {
        "learning_rate": unet_lr,
        "lr_scheduler": lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
        "optimizer_type": "AdamW8bit",
      },
      "training_arguments": {
        "max_train_steps": max_train_steps,
        "max_train_epochs": max_train_epochs,
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "train_batch_size": train_batch_size,
        "noise_offset": None,
        "clip_skip": 2,
        "min_snr_gamma": min_snr_gamma_value,
        "weighted_captions": weighted_captions,
        "seed": 42,
        "max_token_length": 225,
        "xformers": True,
        "lowram": COLAB,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "save_precision": "fp16",
        "mixed_precision": "fp16",
        "output_dir": output_folder,
        "logging_dir": log_folder,
        "output_name": project_name,
        "log_prefix": project_name,
        "save_state": save_state,
        "save_last_n_epochs_state": 1 if save_state else None,
        "resume": last_resume_point
      },
      "model_arguments": {
        "pretrained_model_name_or_path": model_file,
        "v2": custom_model_is_based_on_sd2,
        "v_parameterization": True if custom_model_is_based_on_sd2 else None,
      },
      "saving_arguments": {
        "save_model_as": "safetensors",
      },
      "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
      },
      "dataset_arguments": {
        "cache_latents": True,
      },
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"\n📄 Configuración guardada en {config_file}")

  if override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ Usando configuración de datos propia {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption,
        "keep_tokens": keep_tokens,
        "flip_aug": flip_aug,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
        "min_bucket_reso": 320 if resolution > 640 else 256,
        "max_bucket_reso": 1280 if resolution > 640 else 1024,
      },
      "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "image_dir": images_folder,
              "class_tokens": None if caption_extension else project_name
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"📄 Configuración de datos guardada en {dataset_config_file}")

def download_model():
  global old_model_url, model_url, model_file
  real_model_url = model_url.strip()

  if real_model_url.lower().endswith((".ckpt", ".safetensors")):
    model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
  else:
    model_file = "/content/downloaded_model.safetensors"
    if os.path.exists(model_file):
      !rm "{model_file}"

  if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
    real_model_url = real_model_url.replace("blob", "resolve")
  elif m := re.search(r"(?:https?://)?(?:www\.)?civitai\.com/models/([0-9]+)", model_url):
    real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"

  !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

  if model_file.lower().endswith(".safetensors"):
    from safetensors.torch import load_file as load_safetensors
    try:
      test = load_safetensors(model_file)
      del test
    except Exception as e:
      #if "HeaderTooLarge" in str(e):
      new_model_file = os.path.splitext(model_file)[0]+".ckpt"
      !mv "{model_file}" "{new_model_file}"
      model_file = new_model_file
      print(f"El modelo ahora es {os.path.splitext(model_file)[0]}.ckpt")

  if model_file.lower().endswith(".ckpt"):
    from torch import load as load_ckpt
    try:
      test = load_ckpt(model_file)
      del test
    except Exception as e:
      return False

  return True

def main():
  global dependencies_installed

  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Conectando a Google Drive...")
    drive.mount('/content/drive')

  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return

  if not dependencies_installed:
    print("\n🏭 Instalando...\n")
    t0 = time()
    install_dependencies()
    t1 = time()
    dependencies_installed = True
    print(f"\n✅ Instalación completada en {int(t1-t0)} segundos.")
  else:
    print("\n✅ Ya se ha realizado la instalación.")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔄 Descargando modelo...")
    if not download_model():
      print("\n💥 Error: El modelo que elegiste es inválido o está corrupto, o no se pudo encontrar. Recomiendo usar un enlace de huggingface o civitai.")
      return
    print()
  else:
    print("\n🔄 El modelo ya ha sido descargado.\n")

  create_config()

  print("\n⭐ Iniciando entrenador...\n")
  os.chdir(repo_dir)

  !accelerate launch --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network.py --dataset_config={dataset_config_file} --config_file={config_file}

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    display(Markdown("### ✅ ¡Listo! [Descarga tus Loras desde Google Drive](https://drive.google.com/drive/my-drive)"))

main()


📂 Conectando a Google Drive...
Mounted at /content/drive

💿 Revisando archivos...
📁MyDrive/Loras/jeunji/dataset
📈 Se encontraron 45 imágenes con 20 repeticiones, equivalente a 900 pasos.
📉 Divide 900 pasos en 2 batch size para obtener 450.0 pasos por epoch.
🔮 Habrá 10 epochs, para un total de alrededor de 4500 pasos totales.

🏭 Instalando...

Cloning into '/content/kohya-trainer'...
remote: Enumerating objects: 3342, done.
remote: Counting objects: 100% (1833/1833), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 3342 (delta 1631), reused 1649 (delta 1541), pack-reused 1509
Receiving objects: 100% (3342/3342), 3.59 MiB | 14.02 MiB/s, done.
Resolving deltas: 100% (2321/2321), done.
HEAD is now at 5050971 Merge pull request #388 from kohya-ss/dev
31 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  l

/usr/local/lib/python3.10/dist-packages/safetensors/torch.py:98: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This sh



📄 Configuración guardada en /content/drive/MyDrive/Loras/jeunji/training_config.toml
📄 Configuración de datos guardada en /content/drive/MyDrive/Loras/jeunji/dataset_config.toml

⭐ Iniciando entrenador...

Loading settings from /content/drive/MyDrive/Loras/jeunji/training_config.toml...
/content/drive/MyDrive/Loras/jeunji/training_config
prepare tokenizer
update token length: 225
Load dataset config from /content/drive/MyDrive/Loras/jeunji/dataset_config.toml
prepare images.
found directory /content/drive/MyDrive/Loras/jeunji/dataset contains 45 image files
900 train images with repeating.
0 reg images.
no regularization images / 正則化画像が見つかりませんでした
[Dataset 0]
  batch_size: 2
  resolution: (512, 512)
  enable_bucket: True
  min_bucket_reso: 256
  max_bucket_reso: 1024
  bucket_reso_steps: 64
  bucket_no_upscale: False

  [Subset 0 of Dataset 0]
    image_dir: "/content/drive/MyDrive/Loras/jeunji/dataset"
    image_count: 45
    num_repeats: 20
    shuffle_caption: True
    keep_tokens:

### ✅ ¡Listo! [Descarga tus Loras desde Google Drive](https://drive.google.com/drive/my-drive)

## *️⃣ Extras / Avanzado

Puedes correr estos antes de empezar el entrenamiento.

### 📚 Múltiples carpetas
**Para usuarios avanzados:** Antes de iniciar el entrenamiento, puedes editar y correr la celda aquí abajo, la cual tiene un ejemplo para definir tus propias carpetas de imágenes con diferentes repeticiones.

(El número de repeticiones de la celda principal será ignorado, y también la carpeta principal con el nombre del proyecto)

Puedes hacer que una carpeta contenga imágenes de regularización con la frase `is_reg = true`

In [ ]:
custom_dataset = """
[[datasets]]

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/ejemplo/dataset/imagenes_buenas"
num_repeats = 3

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/ejemplo/dataset/imagenes_normales"
num_repeats = 1

"""

In [ ]:
custom_dataset = None

In [ ]:
#@markdown ### 📂 Extraer datos
#@markdown Es lento subir muchos archivos pequeños, si quieres puedes subir un zip y extraerlo aquí.
zip = "/content/drive/MyDrive/mi_dataset.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/ejemplo/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")


In [ ]:
#@markdown ### 🔢 Contar archivos
#@markdown Google Drive hace imposible contar los archivos en una carpeta, por lo que aquí puedes ver la cantidad de archivos en carpetas y subcarpetas.
carpeta = "/content/drive/MyDrive/Loras" #@param {type:"string"}
folder = carpeta

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
  if tree[path] and others:
    tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))
